## IBSI Chapter 2 Phase 2 − Image filtering

@Author : [MEDomics consortium](https://github.com/medomics/)

@EMAIL : medomics.info@gmail.com

@REF : [IBSI 2](https://www.overleaf.com/read/hwhjswzkhwdh)

### intorduction

The aim of this chapter and this phase is to extract filter-based radiomics features from the same CT-image mentionned in chapter 1. The methodology used to extract these values is described in [IBSI 2 benchmarking 5.2](https://www.overleaf.com/project/5da9e0b82f399f0001ad3970). As shows the following figure, the IBSI chapter 2 gives 2 configurations to test for image processing : configuration *A* and *B*.


<img src="https://github.com/MahdiAll99/MEDimage/blob/main/IBSI-TESTs/images/ibsi2-p2-configurations.png?raw=true" alt="Flowchart of radiomics study"/>

**Note**: The configuration *A* is 2D so it will not be tested (MEDomicsLab does not use slice wise computation).

### Dataset - CT image
We use the same CT image as in IBSI 1 phase 2. The image can be found here: [ibsi_1_ct_radiomics_phantom](https://github.com/theibsi/data_sets/tree/master/ibsi_1_ct_radiomics_phantom)

In [ ]:
import argparse
import math
import os
import pickle
import sys
from copy import deepcopy
from itertools import product
from json import dump, dumps
from pathlib import Path

import matplotlib.pyplot as plt
import nibabel as nib
import numpy as np
from matplotlib.cm import ScalarMappable
from matplotlib.colors import Normalize
from numpyencoder import NumpyEncoder

MODULE_DIR = os.path.dirname(os.path.abspath('../MEDimage/MEDimage.py'))
sys.path.append(os.path.dirname(MODULE_DIR))

import numpy as np
from MEDimage.MEDimage import MEDimage
from MEDimage.MEDimageComputeRadiomics import MEDimageComputeRadiomics
from MEDimage.MEDimageProcessing import MEDimageProcessing
from MEDimage.utils import jsonUtils
from numpyencoder import NumpyEncoder

In [ ]:
from MEDimage.processing.get_roi_from_indexes import get_roi_from_indexes
from MEDimage.processing.roi_extract import roi_extract

In [ ]:
def __getPathResults():
    _rp = Path(os.getcwd()) / "results/ibsi2/phase2"
    if not _rp.exists():
        Path.mkdir(_rp, parents=True)
    return _rp

### Configuration

The first step in this notebook is choosing the test ID (Test to run). Since only configuration B is implemented, we don't specify the configuration in the test ID .i.e. instead of test ID *5.B* we use test ID *5* Test IDs accepted : *1* *2* *3* *4* *5* *6* and *7* (equivalent to *1.B* *2.B* *3.B* *4.B* *5.B* *6.B* and *7.B* in the IBSI). Filters and parameters for the configurations B (ҎD) defined in the following table (The filter parameters for config *B* are in the second row of each test ID):

<img src="https://github.com/MahdiAll99/MEDimage/blob/main/IBSI-TESTs/images/ibsi2-p2-testids.png?raw=true" alt="Flowchart of radiomics study"/>


In [ ]:
test_id = '1' # test ID. More details about tests can be found in the IBSI chapter 2 reference

### Initialization

We start by initializing the important paths to settings folder, dataset folder...

File name should respect the following norm : 
- NPY format : PatientNameOrID__imaging_scan_name.imagning_modality.npy
- NIFTI format : PatientNameOrID__imaging_scan_name(tumorAuto).imagning_modality.nii.gz

In [ ]:
pathData = Path(os.getcwd()) / "data" # Path to the data folder
path_read = pathData / 'CTimage' # Path to the CT-image folder
pathSettings = Path(os.getcwd()) / "settings" # Path to the settings/configuration folder

name_roi = '{GTV-1}' # Region of interest name
name_read = "PAT1__CT(tumorAuto).CTscan.nii.gz" # CT image filename
roi_type = ''

In [ ]:
from MEDimage.processing.interp_volume import interp_volume

def _interpolate(MEDimageProcess, name_roi):
    """
    Runs voxel interpolation on MEDimage volume data and creates the intensity + morphological mask.
    :param MEDimageProcess: Instance of MEDImageProcessing.
    :param name_roi: The name of the region of interest used for the processing
    :return: Two volume objects (Intensity mask and the morphological mask).
    """
    volObjInit, roiObjInit = get_roi_from_indexes(MEDimageProcess, name_roi=name_roi, box_string='full')

    # --------> Intensity Mask :
    vol_obj = interp_volume(
        MEDimageProcess,
        vol_obj_s=volObjInit,
        vox_dim=MEDimageProcess.Params['scaleNonText'],
        interp_met=MEDimageProcess.Params['volInterp'],
        roundVal=MEDimageProcess.Params['glRound'],
        image_type='image',
        roiObjS=roiObjInit,
        box_string=MEDimageProcess.Params['box_string']
    )
    # --------> Morphological Mask :
    roi_obj_morph = interp_volume(
        MEDimageProcess,
        vol_obj_s=roiObjInit,
        vox_dim=MEDimageProcess.Params['scaleNonText'],
        interp_met=MEDimageProcess.Params['roiInterp'],
        roundVal=MEDimageProcess.Params['roiPV'], 
        image_type='roi',
        roiObjS=roiObjInit,
        box_string=MEDimageProcess.Params['box_string']
    )

    return vol_obj, roi_obj_morph

In [ ]:
from MEDimage.processing.range_re_seg import range_re_seg
from MEDimage.processing.outlier_re_seg import outlier_re_seg

def _resegment(MEDimageProcess, vol_obj, roi_obj_morph):
    """
    Outlier resegmentation of the ROI mask.
    :param MEDimageProcess: Instance of MEDImageProcessing.
    :param vol_obj: Filtered volume object.
    :param roi_obj_morph: Morphological mask.
    :return: Volume object of the intensity mask.
    """
    # Range Re-Segmentation :
    roi_obj_int = deepcopy(roi_obj_morph)
    roi_obj_int.data = range_re_seg(vol=vol_obj.data, 
                                 roi=roi_obj_int.data,
                                 im_range=MEDimageProcess.Params['im_range']
    )
    # Outlier Re-Segmentation :
    roi_obj_int.data = np.logical_and(
        outlier_re_seg(vol=vol_obj.data, roi=roi_obj_int.data, outliers=MEDimageProcess.Params['outliers']),
        roi_obj_int.data
    ).astype(int)
    
    return roi_obj_int

In [ ]:
def plot_comparison(result, original_data, _slice, test_id, filter_name):
    """
    Plot the coronal, axial and sagittal slices of the original image and the result image.
    :param result: The result obtained by the program.
    :param original_data: The original image
    :param _slice: Which slice will be plotted along each axis.
    :param test_id: The test identifier as string that will be used in the figure name. "Exemple: 2"
    :param filter_name: Name of the filter used.
    """
    if not filter_name:
        filter_name = 'no filter'
        
    if len(original_data.shape) > 3:
        original_data = np.squeeze(original_data)

    fig = plt.figure(figsize=(12, 12))
    fig.canvas.manager.set_window_title('Phase2_ID' + test_id + 'B')

    if original_data is not None:

        fig.suptitle(f'Original image vs Processed image using {filter_name}. (Test ID : {test_id}.B)', fontsize=16)

        fig.add_subplot(2, 3, 1, ylabel="Original image", title="Coronal")
        plt.imshow(original_data[:, :, _slice], cmap='gray')

        fig.add_subplot(2, 3, 2, title="Axial")
        plt.imshow(original_data[:, _slice, :], cmap='gray')

        fig.add_subplot(2, 3, 3, title="Sagittal")
        plt.imshow(original_data[_slice, :, :], cmap='gray')

        fig.add_subplot(2, 3, 4, ylabel="Result")
        plt.imshow(result[:, :, _slice], cmap='gray')

        fig.add_subplot(2, 3, 5)
        plt.imshow(result[:, _slice, :], cmap='gray')

        fig.add_subplot(2, 3, 6)
        plt.imshow(result[_slice, :, :], cmap='gray')

    else:
        fig.add_subplot(1, 3, 1, ylabel="Result")
        plt.imshow(result[0, :, :, _slice], cmap='gray')

        fig.add_subplot(1, 3, 2)
        plt.imshow(result[0, :, _slice, :], cmap='gray')

        fig.add_subplot(1, 3, 3)
        plt.imshow(result[0, _slice, :, :], cmap='gray')

    plt.show()

In [ ]:
def save_results(pathResults, test_id, DiagFeatures, int_hist, stats):
    """
    Saves the results in a JSON file under the formatting : Results_P2.{test_id}B.json
    :param test_id: The test identifier as string. "Exemple: 2"
    :param int_hist: Dict of Intensity Histogram features.
    :param stats: Dict of Statistical features.
    """

    results = { 'Diagnostics' : DiagFeatures,
                'intHist_3D' : int_hist,
                'stats_3D' : stats}

    # save results in json file:
    with open(pathResults / f"Results_P2.{test_id}B.json", "w") as fp:   
        dump(results, fp, indent=4, cls=NumpyEncoder)

### Initilization (continued)

As mentioned before, CT-scan have a voxel spacing of 1 so it's gonna be our voxel length and will be used to compute the kernel size for some filter.

In [ ]:
VOLEX_LENGTH = 1

Extract the right parameters/configurations for the script

In [ ]:
im_params = jsonUtils.load_json(pathSettings / 'IBSI2Phase2B_settings.json')

In this notebook we are going to use the **MEDimage** class and its child **MEDimageProcessing** to filter the images. So the first step is to initialize the MEDimage class using a **NIFTI** file.

In [ ]:
from MEDimage.utils.initMEDimage import initMEDimage

MEDimageProcess, MEDimageCR = initMEDimage(name_read, path_read, roi_type, im_params, 'log_file_ibsi2p2.txt')

### Image processing

Processing is done prior to image filtering. The processing steps are:
- Segmentation (Creation of ROI mask)
- Interpolation :
    - resampled voxel spacing (mm) : [1 × 1 × 1]
    - interpolation method : tricubic spline
    - intensity rounding : nearest integer
    - ROI interpolation method : trilinear
    - ROI partial mask volume : 0.5
- Re-segmentation :
    - range(HU) : [-1000, 400]
- Image filtering
- ROI extraction

**PS**: We assume that the IBSI chapter 1 is tested and the image processing steps are now clear, so no details are given here.

In [ ]:
volObjInit, roiObjInit = get_roi_from_indexes(MEDimageProcess, name_roi=name_roi, box_string='full')

#### Diagnostic features
The diagnostic features are computed before and after re-segmentation and interpolation to identify the issues with the implementation (if there is any).

##### Initial diagnostics

In [ ]:
from MEDimage.processing.get_diag_features import get_diag_features

# Extract initial diagnostic featues
DIAG_init = get_diag_features(volObjInit, roiObjInit, roiObjInit, 'initial')

#### Interpolation

In [ ]:
# Interpolate
vol_obj, roi_obj_morph = _interpolate(MEDimageProcess, name_roi)

We compare voxel spacing before and after interpolation

In [ ]:
# voxel spacing before interpolation
print('Before interp:', volObjInit.spatial_ref.PixelExtentInWorldX, 
      volObjInit.spatial_ref.PixelExtentInWorldY,
      volObjInit.spatial_ref.PixelExtentInWorldZ)
# voxel spacing after interpolation
print('After interp:',vol_obj.spatial_ref.PixelExtentInWorldX, 
      vol_obj.spatial_ref.PixelExtentInWorldY,
      vol_obj.spatial_ref.PixelExtentInWorldZ)
# Desired voxel spacing 
print('Desired voxel spacing is [1, 1, 1]')

#### Re-segmentation

In [ ]:
roi_obj_int = _resegment(MEDimageProcess, vol_obj, roi_obj_morph)

##### Post-processing diagnostics

In [ ]:
# Extract Diagnostic features after interpolation and re-segmentation
DIAG_reSeg = get_diag_features(vol_obj, roi_obj_int, roi_obj_morph, 'reSeg')

### Image filtering

Unlike the phase 1, we use the MEDimage method ***applyFilter()*** to filter the CT image. The method uses the same process as phase 1. 

The parameters needed are the filter name/type and the image volume object.

In [ ]:
test_id = str(test_id)
filter_name = ''

if test_id == "1":
    filter_name = ''

elif test_id == "2":
    filter_name = 'Mean'

elif test_id == "3":
    filter_name = 'LoG'

elif test_id == "4":
    filter_name = 'Laws'

elif test_id == "5":
    filter_name = 'Gabor'

elif test_id == "6":
    filter_name = 'Wavelet_db3_LLH'

elif test_id == "7":        
    filter_name = 'Wavelet_db3_HHH'

else:
    raise NotImplementedError

if filter_name:
    vol_obj = MEDimageProcess.applyFilter(filter_name, vol_obj)

#### ROI-Extraction

In [ ]:
vol_int_re = roi_extract(
    vol=vol_obj.data, 
    roi=roi_obj_int.data
)

#### Features computation

As mentioned in the IBSI, only part of the radiomics features standardized previously is gonna be computed. The features computed are : *statistical features* and *intensity histogram features*.

In [ ]:
# imports
from MEDimage.biomarkers.getIntHistFeatures import getIntHistFeatures
from MEDimage.biomarkers.getStatsFeatures import getStatsFeatures

# Preparation of computation :
MEDimageCR.init_nft_calculation(vol_obj)

# Extract statistical and intenisty-histogram features
# Intensity Histogram Features
int_hist = getIntHistFeatures(
        vol=vol_int_re
    )

# Stats Features
Stats = getStatsFeatures(
        vol=vol_int_re,
        intensity=MEDimageCR.Params['intensity']
    )

# Diagnostics Features
DiagFeatures = {
    'diag_n_voxel' : DIAG_init['roi_initial_Int_voxNumb'],
    'diag_n_voxel_interp_reseg' : DIAG_reSeg['roi_reSeg_Int_voxNumb'],
    'diag_mean_int_interp_reseg' : DIAG_reSeg['roi_reSeg_meanInt'],
    'diag_max_int_interp_reseg' : DIAG_reSeg['roi_reSeg_maxInt'],
    'diag_min_int_interp_reseg' : DIAG_reSeg['roi_reSeg_minInt']
    }

Print the features extraction results

In [ ]:
print(dumps(
    {'Diagnostics' : DiagFeatures, 
     'intHist_3D' : int_hist, 
     'stats_3D' : Stats}, 
    indent=4, 
    cls=NumpyEncoder)
     )

Finally we plot the before-and-after image filtering

In [ ]:
_slice = 31
plot_comparison(vol_obj.data, volObjInit.data, _slice=_slice, test_id=test_id, filter_name=filter_name)

Run this cell to save the results in JSON format on your machine

In [ ]:
#pathResults = __getPathResults() # Path to where the results are gonna be saved
#save_results(pathResults, test_id, DiagFeatures, int_hist, Stats)

The IBSI chapter 2 phase 2 features calculation results for *Udes* Team have been already submitted and can be found here : [Latest submissions](https://ibsi.radiomics.hevs.ch/#ibsi2-phase2-pane).